## Calculate biomass with GEE JS

  Documentation on how to create a microservice with GE engine. In this specific case it 
  calculates biomass and biomass density paralelly for all admin 2 regions in the GADM table
  by using the map() function. It can be used as a model to calculate other attributes. 
  We start off by getting the GADM admin 2 areas from https://gadm.org/download_world.html (click on download as six separate layers, then take only level_2). To preprocess the data, some simple python code is used to get rid of unnecessary columns.
  ```python 
df = gpd.read_file('/Users/nrigheriu/vizzuality/data/gadm36_level2_shp/gadm36_2.shp', encoding='utf-8')
iso_col = ['blah'] * len(df)
df['iso'] = iso_col
df['admin_1'] = iso_col
df['admin_2'] = iso_col
def process_gid_2(gid_2):
    """Return dict of iso (string), and admin_1 and admin_2 (ints) from gid_2 entry."""
    try:
        iso, admin_1, tmp_admin_2 = gid_2.split('.')
        admin_2 = tmp_admin_2.split('_')[0]
        return {'iso':iso, 'admin_1':int(admin_1), 'admin_2':int(admin_2)}
    except:
        return {'iso':"", 'admin_1':"", 'admin_2':""}
gid_split = [process_gid_2(df['GID_2'][i]) for i in range(len(df['GID_2']))]
df['iso'] = [gid_split[i]['iso'] for i in range(len(gid_split))] 
df['admin_1'] = [gid_split[i]['admin_1'] for i in range(len(gid_split))] 
df['admin_2'] = [gid_split[i]['admin_2'] for i in range(len(gid_split))] 
out_df = gpd.GeoDataFrame(df[['GID_2','iso', 'admin_1', 'admin_2']],geometry=df.geometry, crs=df.crs)
!mkdir world_parsed_gid
out_df.to_file(driver = 'ESRI Shapefile', filename='world_parsed_gid/world_parsed_gid.shp')
```
Afterwards the shapefile is inserted in http://mapshaper.org/ and the features are simplified to about 1% to make sure it will be accepted in earth engine when uploading it as a table (here it's named gadm36).
  Beware the script takes several hours (>4 hrs) to complete. For smaller scale experiments, try using a data set like regions of Spain first. 
  Firstly start off by importing the gadm table containing admin level geometries and the 
  other 2 tables whrc_carbon and forest_change to help in calculating biomass. 
```javascript
var whrc_carbon = ee.ImageCollection("projects/wri-datalab/WHRC_CARBON"),
    forest_change = ee.Image("projects/wri-datalab/HansenComposite_17"),
    gadm36 = ee.FeatureCollection("projects/wri-datalab/gadm36");

```
  The function addBiomass() has a threshold parameter which corresponds to the dataset 
  being passed (e.g. thresholdVal 10 for dataset th10) and adds a threshold property as 
  well as biomass and biomassDensity density corresponding to this threshold value 
  for all the entries in gadm36 (which consists of admin2 areas). 
  After adding the threshold property,  a mask with tree cover is created with help 
  of the forest_change data. 
  The addBiomassSum function gets the biomass_masked property and uses the Reducer function to create 
  a new attribute called biomass. It does so with the feature.set() function. 
  Afterwards, the biomassSum attribute is stored in a variable and used to calculate and add a new attribute
  called biomassDensity which is the biomassSum divided by the area. Finally the geometry of the features is 
  deleted by setting it to null because it isn't needed anymore.
  
  NOTE: feature properties/attributes cannot be printed inside the map() function because it is executed 
  in paralell on different machines. The code has to be written and trusted that it works or debugged in 
  a different way.   
```javascript
var addBiomass = function(thresholdVal) {
   var addThreshold = function(feature) {
    return feature.set({threshold: thresholdVal});
  };
  var addArea = function(feature) {
    return feature.set({areaHa: feature.geometry().area().divide(100 * 100)});
  };
  var dataset = gadm36.map(addThreshold);
  dataset = dataset.map(addArea);
  var tree_mask = forest_change.select('tree_' + thresholdVal).gt(0);
  var biomass_masked = whrc_carbon.max().multiply(ee.Image.pixelArea().divide(10000)).mask(tree_mask);
  var addBiomassSum = function(feature) {
    feature = feature.set({biomass: biomass_masked.reduceRegion({
      reducer: ee.Reducer.sum().unweighted(),
      geometry: feature.geometry(),
      bestEffort: true,
      scale:30 }).get('b1')});              //The actual calculated biomass is in the attribute b1
    var areaHa = ee.Number(feature.get('areaHa'));
    var biomassSum = ee.Number(feature.get('biomass'));
    feature = feature.set({biomassDensity: biomassSum.divide(areaHa)})
                      .setGeometry(null); 
    return feature; 
  };
  dataset = dataset.map(addBiomassSum);
  return dataset;
}; 
```
   To have a final table with each original values duplicated for each of the threshold values,
   multiple datasets are created and merged. Finally, the table will consist of 7 times 
   more rows (as there are 7 different threshold values), where the extra added rows contain
   the same data as the original ones, except the values indicating threshold, 
   biomass and biomassDensity.    
```javascript
var th10 = addBiomass(10);
var th15 = addBiomass(15);
var th20 = addBiomass(20);
var th25 = addBiomass(25);
var th30 = addBiomass(30);
var th50 = addBiomass(50);
var th75 = addBiomass(75);
var final_table = th10.merge(th15);
final_table = final_table.merge(th20);
final_table = final_table.merge(th25);
final_table = final_table.merge(th30);
final_table = final_table.merge(th50);
final_table = final_table.merge(th75);
```
The table needs to be exported, and not printed because it is too large to be printed. 
```javascript
//print(gadm36);  //don't print or else it'll cause scaling errors
Export.table.toCloudStorage({
  collection: final_table,
  description: 'gadm36BiomassTHs',
  fileFormat: 'CSV'
});
```